# 从NCBI下载所有C. glutamicum基因组

In [1]:
#!/usr/bin/env python3
"""
NCBI Datasets 分批下载脚本(优化版)
用于下载 Corynebacterium glutamicum 的基因组数据
特性:
- 进度条显示
- 单个基因组级别的重试
- 断点续传
- 详细日志记录
- 只下载 RefSeq (GCF) 基因组
- 包含 CDS 序列
"""

import subprocess
import json
import os
import time
import logging
from pathlib import Path
from datetime import datetime
from typing import List, Tuple

try:
    from tqdm import tqdm
except ImportError:
    print("警告: 未安装 tqdm 库,将使用简单进度显示")
    print("建议安装: pip install tqdm")
    tqdm = None


class DownloadLogger:
    """日志管理器"""
    def __init__(self, log_dir: str = "logs"):
        Path(log_dir).mkdir(exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = os.path.join(log_dir, f"download_{timestamp}.log")

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file, encoding='utf-8'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
        self.logger.info(f"日志文件: {log_file}")


def get_genome_accessions(taxon_name: str,
                          assembly_source: str = "refseq",
                          assembly_level: str = "complete") -> List[str]:
    """获取基因组列表"""
    logger = logging.getLogger(__name__)
    logger.info(f"正在获取 {taxon_name} 的基因组列表...")
    logger.info(f"数据来源: {assembly_source}")
    logger.info(f"组装级别: {assembly_level}")

    cmd = [
        "datasets", "summary", "genome", "taxon", taxon_name,
        "--assembly-source", assembly_source,
        "--assembly-level", assembly_level,
        "--as-json-lines"
    ]

    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        accessions = []

        for line in result.stdout.strip().split('\n'):
            if line:
                data = json.loads(line)
                accession = data.get('accession')
                if accession:
                    accessions.append(accession)

        logger.info(f"找到 {len(accessions)} 个 {assembly_source.upper()} 基因组")
        return accessions

    except subprocess.CalledProcessError as e:
        logger.error(f"无法获取基因组列表: {e.stderr}")
        return []


def check_existing_downloads(output_dir: str) -> set:
    """检查已下载的批次"""
    existing = set()
    if not os.path.exists(output_dir):
        return existing

    for filename in os.listdir(output_dir):
        if filename.startswith("cglutamicum_batch_") and filename.endswith(".zip"):
            # 提取批次号
            batch_num = int(filename.split("_")[2].split(".")[0])
            # 检查文件大小(至少1KB才算有效)
            filepath = os.path.join(output_dir, filename)
            if os.path.getsize(filepath) > 1024:
                existing.add(batch_num)

    return existing


def download_single_genome(accession: str, output_dir: str, include_types: str,
                          assembly_source: str = "refseq",
                          max_retries: int = 3) -> bool:
    """
    下载单个基因组(带重试)

    参数:
        accession: 基因组登录号
        output_dir: 输出目录
        include_types: 要下载的数据类型
        assembly_source: 数据来源
        max_retries: 最大重试次数
    """
    logger = logging.getLogger(__name__)
    filename = f"{accession}.zip"
    filepath = os.path.join(output_dir, filename)

    # 如果文件已存在且大小>1KB,跳过
    if os.path.exists(filepath) and os.path.getsize(filepath) > 1024:
        logger.debug(f"跳过已存在的文件: {accession}")
        return True

    cmd = [
        "datasets", "download", "genome", "accession",
        accession,
        "--include", include_types,
        "--assembly-source", assembly_source,  # 指定数据来源
        "--filename", filepath
    ]

    for attempt in range(max_retries):
        try:
            subprocess.run(cmd, check=True, capture_output=True)
            logger.debug(f"✓ {accession} 下载成功")
            return True
        except subprocess.CalledProcessError as e:
            if attempt < max_retries - 1:
                logger.warning(f"✗ {accession} 下载失败 (尝试 {attempt + 1}/{max_retries})")
                time.sleep(2)
            else:
                logger.error(f"✗ {accession} 下载失败 (已重试{max_retries}次)")
                return False

    return False


def download_batch(accessions: List[str], batch_num: int, output_dir: str,
                   include_types: str, assembly_source: str = "refseq",
                   max_retries: int = 3) -> Tuple[int, List[str]]:
    """
    下载一批基因组(逐个下载以支持断点续传)
    返回: (成功数量, 失败的accessions列表)
    """
    logger = logging.getLogger(__name__)
    batch_dir = os.path.join(output_dir, f"batch_{batch_num:03d}")
    Path(batch_dir).mkdir(exist_ok=True)

    logger.info(f"\n{'='*60}")
    logger.info(f"批次 {batch_num}: {len(accessions)} 个基因组")
    logger.info(f"保存到: {batch_dir}/")

    successful = 0
    failed_accessions = []

    # 使用进度条
    iterator = tqdm(accessions, desc=f"批次 {batch_num}",
                   unit="genome", ncols=100) if tqdm else accessions

    for accession in iterator:
        if download_single_genome(accession, batch_dir, include_types,
                                  assembly_source, max_retries):
            successful += 1
        else:
            failed_accessions.append(accession)

        # 如果不使用tqdm,手动打印进度
        if not tqdm:
            print(f"  进度: {successful}/{len(accessions)} (失败: {len(failed_accessions)})")

    # 如果全部成功,创建批次标记文件
    if not failed_accessions:
        marker_file = os.path.join(output_dir, f"batch_{batch_num:03d}_complete.marker")
        with open(marker_file, 'w') as f:
            f.write(f"Completed at {datetime.now()}\n")
            f.write(f"Total genomes: {len(accessions)}\n")

    logger.info(f"批次 {batch_num} 完成: {successful}/{len(accessions)} 成功")

    return successful, failed_accessions


def retry_failed_downloads(failed_map: dict, output_dir: str, include_types: str,
                          assembly_source: str = "refseq") -> dict:
    """重试所有失败的下载"""
    logger = logging.getLogger(__name__)

    if not failed_map:
        return {}

    logger.info(f"\n{'='*60}")
    logger.info(f"开始重试失败的下载...")
    logger.info(f"失败批次数: {len(failed_map)}")
    total_failed = sum(len(accs) for accs in failed_map.values())
    logger.info(f"失败基因组总数: {total_failed}")

    still_failed = {}

    for batch_num, accessions in failed_map.items():
        logger.info(f"\n重试批次 {batch_num}: {len(accessions)} 个基因组")
        batch_dir = os.path.join(output_dir, f"batch_{batch_num:03d}")

        successful, failed = download_batch(accessions, batch_num, output_dir,
                                           include_types, assembly_source,
                                           max_retries=5)

        if failed:
            still_failed[batch_num] = failed

    return still_failed


def save_failed_list(failed_map: dict, output_dir: str):
    """保存失败列表到文件"""
    if not failed_map:
        return

    failed_file = os.path.join(output_dir, "failed_accessions.txt")
    with open(failed_file, 'w') as f:
        f.write(f"下载失败的基因组列表 - {datetime.now()}\n")
        f.write("="*60 + "\n\n")

        for batch_num, accessions in sorted(failed_map.items()):
            f.write(f"批次 {batch_num}:\n")
            for acc in accessions:
                f.write(f"  {acc}\n")
            f.write("\n")

    logging.getLogger(__name__).info(f"失败列表已保存到: {failed_file}")


def print_summary(total_genomes: int, total_batches: int,
                 successful_batches: int, failed_map: dict,
                 assembly_source: str, include_types: str):
    """打印下载摘要"""
    logger = logging.getLogger(__name__)

    total_failed = sum(len(accs) for accs in failed_map.values())
    total_successful = total_genomes - total_failed

    logger.info("\n" + "="*60)
    logger.info("下载完成!")
    logger.info(f"数据来源: {assembly_source.upper()}")
    logger.info(f"数据类型: {include_types}")
    logger.info(f"总基因组数: {total_genomes}")
    logger.info(f"成功下载: {total_successful} ({total_successful/total_genomes*100:.1f}%)")
    logger.info(f"下载失败: {total_failed} ({total_failed/total_genomes*100:.1f}%)")
    logger.info(f"成功批次: {successful_batches}/{total_batches}")

    if failed_map:
        logger.info(f"\n失败的批次: {', '.join(map(str, sorted(failed_map.keys())))}")
        logger.info(f"详细信息请查看: failed_accessions.txt")

    logger.info("="*60)


def main():
    # ==================== 配置参数 ====================
    TAXON_NAME = "Corynebacterium glutamicum"
    BATCH_SIZE = 30
    OUTPUT_DIR = "cglutamicum_downloads"

    # 👇 修改1: 添加 cds 到下载类型
    INCLUDE_TYPES = "genome,protein,gff3,cds"

    # 👇 修改2: 只下载 RefSeq (GCF) 基因组
    # 可选值: "refseq" (只下载GCF), "genbank" (只下载GCA), "all" (全部下载)
    ASSEMBLY_SOURCE = "refseq"
    ASSEMBLY_LEVEL = "complete"
    MAX_RETRIES = 3
    ENABLE_RESUME = True  # 启用断点续传
    # =================================================

    # 初始化日志
    log_manager = DownloadLogger()
    logger = logging.getLogger(__name__)

    logger.info(f"开始下载 {TAXON_NAME}")
    logger.info(f"批次大小: {BATCH_SIZE}")
    logger.info(f"输出目录: {OUTPUT_DIR}")
    logger.info(f"数据类型: {INCLUDE_TYPES}")
    logger.info(f"数据来源: {ASSEMBLY_SOURCE} (GCF=RefSeq, GCA=GenBank)")
    logger.info(f"断点续传: {'启用' if ENABLE_RESUME else '禁用'}")

    # 创建输出目录
    Path(OUTPUT_DIR).mkdir(exist_ok=True)

    # 获取所有基因组登录号（只获取 RefSeq）
    accessions = get_genome_accessions(TAXON_NAME, ASSEMBLY_SOURCE,ASSEMBLY_LEVEL)

    if not accessions:
        logger.error("未找到基因组数据,退出")
        return

    # 显示获取到的基因组前缀统计
    gcf_count = sum(1 for acc in accessions if acc.startswith("GCF"))
    gca_count = sum(1 for acc in accessions if acc.startswith("GCA"))
    logger.info(f"基因组统计: GCF={gcf_count}, GCA={gca_count}")

    # 检查已完成的批次
    completed_batches = set()
    if ENABLE_RESUME:
        for filename in os.listdir(OUTPUT_DIR):
            if filename.endswith("_complete.marker"):
                batch_num = int(filename.split("_")[1])
                completed_batches.add(batch_num)

        if completed_batches:
            logger.info(f"发现已完成的批次: {sorted(completed_batches)}")

    # 计算批次数
    total_batches = (len(accessions) + BATCH_SIZE - 1) // BATCH_SIZE
    logger.info(f"\n将分 {total_batches} 批次下载")

    # 分批下载
    successful_batches = 0
    failed_map = {}  # {batch_num: [failed_accessions]}

    # 主进度条
    batch_iterator = range(0, len(accessions), BATCH_SIZE)
    if tqdm:
        batch_iterator = tqdm(list(batch_iterator), desc="总进度", unit="批次", ncols=100)

    for i in batch_iterator:
        batch_num = (i // BATCH_SIZE) + 1

        # 跳过已完成的批次
        if batch_num in completed_batches:
            logger.info(f"跳过已完成的批次 {batch_num}")
            successful_batches += 1
            continue

        batch_accessions = accessions[i:i + BATCH_SIZE]

        successful, failed = download_batch(
            batch_accessions, batch_num, OUTPUT_DIR,
            INCLUDE_TYPES, ASSEMBLY_SOURCE, MAX_RETRIES
        )

        if not failed:
            successful_batches += 1
        else:
            failed_map[batch_num] = failed

        # 批次间延迟
        if i + BATCH_SIZE < len(accessions):
            time.sleep(1)

    # 重试失败的下载
    if failed_map:
        logger.info(f"\n发现 {len(failed_map)} 个批次有失败的下载")
        response = input("是否重试失败的下载? (y/n): ").strip().lower()

        if response == 'y':
            failed_map = retry_failed_downloads(failed_map, OUTPUT_DIR,
                                                INCLUDE_TYPES, ASSEMBLY_SOURCE)

    # 保存失败列表
    if failed_map:
        save_failed_list(failed_map, OUTPUT_DIR)

    # 打印摘要
    print_summary(len(accessions), total_batches, successful_batches,
                  failed_map, ASSEMBLY_SOURCE, INCLUDE_TYPES)

    logger.info(f"\n所有文件保存在: {OUTPUT_DIR}/")
    logger.info(f"\n下载的文件类型说明:")
    logger.info(f"  - genome: 基因组序列 (*_genomic.fna)")
    logger.info(f"  - protein: 蛋白质序列 (*_protein.faa)")
    logger.info(f"  - gff3: GFF3注释文件 (*_genomic.gff)")
    logger.info(f"  - cds: CDS核苷酸序列 (*_cds_from_genomic.fna)")

if __name__ == "__main__":
    main()

2025-12-19 10:15:26,413 - INFO - 日志文件: logs\download_20251219_101526.log
2025-12-19 10:15:26,414 - INFO - 开始下载 Corynebacterium glutamicum
2025-12-19 10:15:26,415 - INFO - 批次大小: 30
2025-12-19 10:15:26,415 - INFO - 输出目录: cglutamicum_downloads
2025-12-19 10:15:26,415 - INFO - 数据类型: genome,protein,gff3,cds
2025-12-19 10:15:26,416 - INFO - 数据来源: refseq (GCF=RefSeq, GCA=GenBank)
2025-12-19 10:15:26,417 - INFO - 断点续传: 启用
2025-12-19 10:15:26,417 - INFO - 正在获取 Corynebacterium glutamicum 的基因组列表...
2025-12-19 10:15:26,418 - INFO - 数据来源: refseq
2025-12-19 10:15:26,418 - INFO - 组装级别: complete
2025-12-19 10:15:30,720 - INFO - 找到 34 个 REFSEQ 基因组
2025-12-19 10:15:30,720 - INFO - 基因组统计: GCF=34, GCA=0
2025-12-19 10:15:30,721 - INFO - 
将分 2 批次下载
总进度:   0%|                                                               | 0/2 [00:00<?, ?批次/s]2025-12-19 10:15:30,734 - INFO - 
2025-12-19 10:15:30,734 - INFO - 批次 1: 30 个基因组
2025-12-19 10:15:30,735 - INFO - 保存到: cglutamicum_downloads\batch_001/

批次 1: 100%|████

# 解压

In [2]:
#!/usr/bin/env python3
"""
解压和整理从NCBI下载的基因组数据
匹配 download_cglutamicum.py 的输出结构
"""

import os
import zipfile
import shutil
import json
from pathlib import Path
from tqdm import tqdm
import logging
from datetime import datetime

class GenomeExtractor:
    """基因组数据提取器"""

    def __init__(self, download_dir="cglutamicum_downloads", output_dir="cglutamicum_pangenome"):
        self.download_dir = download_dir
        self.output_dir = output_dir

        # 创建输出目录结构
        self.dirs = {
            'genomes': os.path.join(output_dir, 'genomes'),
            'proteins': os.path.join(output_dir, 'proteins'),
            'cds': os.path.join(output_dir, 'cds'),  # 👈 新增 CDS 目录
            'gff': os.path.join(output_dir, 'gff'),
            'metadata': os.path.join(output_dir, 'metadata'),
            'logs': os.path.join(output_dir, 'logs')
        }

        for dir_path in self.dirs.values():
            Path(dir_path).mkdir(parents=True, exist_ok=True)

        # 设置日志
        self._setup_logging()

    def _setup_logging(self):
        """设置日志"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = os.path.join(self.dirs['logs'], f"extract_{timestamp}.log")

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file, encoding='utf-8'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
        self.logger.info(f"日志文件: {log_file}")

    def find_all_genome_zips(self):
        """查找所有下载的基因组zip文件"""
        self.logger.info(f"扫描下载目录: {self.download_dir}")

        genome_zips = []

        # 遍历所有batch目录
        if not os.path.exists(self.download_dir):
            self.logger.error(f"下载目录不存在: {self.download_dir}")
            return []

        for item in os.listdir(self.download_dir):
            item_path = os.path.join(self.download_dir, item)

            # 查找 batch_xxx 目录
            if os.path.isdir(item_path) and item.startswith('batch_'):
                # 在batch目录中查找所有.zip文件
                for zip_file in os.listdir(item_path):
                    if zip_file.endswith('.zip'):
                        zip_path = os.path.join(item_path, zip_file)
                        # 提取accession号
                        accession = zip_file.replace('.zip', '')
                        genome_zips.append({
                            'accession': accession,
                            'path': zip_path,
                            'batch': item
                        })

        self.logger.info(f"找到 {len(genome_zips)} 个基因组文件")
        return genome_zips

    def extract_single_genome(self, genome_info):
        """
        解压单个基因组并提取关键文件
        返回: (成功, 元数据字典)
        """
        accession = genome_info['accession']
        zip_path = genome_info['path']

        # 检查文件大小
        if os.path.getsize(zip_path) < 1024:
            self.logger.warning(f"文件过小,可能损坏: {accession}")
            return False, None

        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # 查找并提取文件
                extracted_files = {
                    'genome': None,  # .fna
                    'protein': None, # .faa
                    'cds': None,     # _cds_from_genomic.fna  👈 新增
                    'gff': None,     # .gff
                    'jsonl': None    # data_report.jsonl
                }

                for file_info in zip_ref.filelist:
                    filename = file_info.filename

                    # CDS 核苷酸序列文件 👈 新增（放在基因组之前判断）
                    if 'cds_from_genomic.fna' in filename:
                        target = os.path.join(self.dirs['cds'], f"{accession}_cds.fna")
                        with zip_ref.open(filename) as source:
                            with open(target, 'wb') as dest:
                                shutil.copyfileobj(source, dest)
                        extracted_files['cds'] = target

                    # 基因组序列文件（排除 cds 和 rna）
                    elif (filename.endswith('.fna') or filename.endswith('_genomic.fna')) and \
                         'cds_from_genomic' not in filename and 'rna_from_genomic' not in filename:
                        target = os.path.join(self.dirs['genomes'], f"{accession}.fna")
                        with zip_ref.open(filename) as source:
                            with open(target, 'wb') as dest:
                                shutil.copyfileobj(source, dest)
                        extracted_files['genome'] = target

                    # 蛋白质序列文件
                    elif filename.endswith('.faa') or filename.endswith('_protein.faa'):
                        target = os.path.join(self.dirs['proteins'], f"{accession}.faa")
                        with zip_ref.open(filename) as source:
                            with open(target, 'wb') as dest:
                                shutil.copyfileobj(source, dest)
                        extracted_files['protein'] = target

                    # GFF注释文件
                    elif filename.endswith('.gff') or filename.endswith('_genomic.gff'):
                        target = os.path.join(self.dirs['gff'], f"{accession}.gff")
                        with zip_ref.open(filename) as source:
                            with open(target, 'wb') as dest:
                                shutil.copyfileobj(source, dest)
                        extracted_files['gff'] = target

                    # 元数据
                    elif 'data_report.jsonl' in filename:
                        with zip_ref.open(filename) as source:
                            content = source.read().decode('utf-8')
                            extracted_files['jsonl'] = content

                # 解析元数据
                metadata = self._parse_metadata(accession, extracted_files)

                # 验证关键文件
                if not extracted_files['genome']:
                    self.logger.warning(f"缺少基因组序列文件: {accession}")
                    return False, None

                return True, metadata

        except Exception as e:
            self.logger.error(f"解压失败 {accession}: {str(e)}")
            return False, None

    def _parse_metadata(self, accession, extracted_files):
        """解析基因组元数据"""
        metadata = {
            'accession': accession,
            'organism': 'Corynebacterium glutamicum',
            'has_genome': extracted_files['genome'] is not None,
            'has_protein': extracted_files['protein'] is not None,
            'has_cds': extracted_files['cds'] is not None,  # 👈 新增
            'has_gff': extracted_files['gff'] is not None,
        }

        # 从JSONL解析更多信息
        if extracted_files['jsonl']:
            try:
                data = json.loads(extracted_files['jsonl'])
                metadata.update({
                    'organism_name': data.get('organism', {}).get('organismName', 'N/A'),
                    'strain': data.get('organism', {}).get('infraspecificNames', {}).get('strain', 'N/A'),
                    'assembly_level': data.get('assemblyInfo', {}).get('assemblyLevel', 'N/A'),
                    'contig_n50': data.get('assemblyStats', {}).get('contigN50', 'N/A'),
                    'total_sequence_length': data.get('assemblyStats', {}).get('totalSequenceLength', 'N/A'),
                    'number_of_contigs': data.get('assemblyStats', {}).get('numberOfContigs', 'N/A'),
                })
            except Exception as e:
                self.logger.debug(f"无法解析元数据: {accession}")

        return metadata

    def extract_all_genomes(self, genome_zips):
        """提取所有基因组"""
        self.logger.info("开始提取基因组文件...")

        successful = 0
        failed = []
        all_metadata = []

        # 使用进度条
        for genome_info in tqdm(genome_zips, desc="解压基因组", unit="genome"):
            success, metadata = self.extract_single_genome(genome_info)

            if success:
                successful += 1
                if metadata:
                    all_metadata.append(metadata)
            else:
                failed.append(genome_info['accession'])

        self.logger.info(f"\n提取完成: {successful}/{len(genome_zips)} 成功")

        if failed:
            self.logger.warning(f"失败: {len(failed)} 个")
            failed_file = os.path.join(self.dirs['logs'], 'extraction_failed.txt')
            with open(failed_file, 'w') as f:
                f.write('\n'.join(failed))
            self.logger.info(f"失败列表: {failed_file}")

        # 保存元数据摘要
        self._save_metadata_summary(all_metadata)

        return successful, failed

    def _save_metadata_summary(self, metadata_list):
        """保存元数据摘要"""
        if not metadata_list:
            return

        import pandas as pd

        df = pd.DataFrame(metadata_list)

        # 保存为CSV
        csv_file = os.path.join(self.dirs['metadata'], 'genomes_summary.csv')
        df.to_csv(csv_file, index=False)
        self.logger.info(f"元数据摘要: {csv_file}")

        # 打印统计
        self.logger.info("\n=== 基因组统计 ===")
        self.logger.info(f"总数: {len(df)}")
        self.logger.info(f"有GFF注释: {df['has_gff'].sum()}")
        self.logger.info(f"有蛋白序列: {df['has_protein'].sum()}")
        self.logger.info(f"有CDS序列: {df['has_cds'].sum()}")  # 👈 新增

        if 'assembly_level' in df.columns:
            self.logger.info("\n组装水平分布:")
            for level, count in df['assembly_level'].value_counts().items():
                self.logger.info(f"  {level}: {count}")

    def generate_file_lists(self):
        """生成文件列表(用于后续分析)"""
        lists = {
            'genomes': [],
            'proteins': [],
            'cds': [],  # 👈 新增
            'gff': []
        }

        for key, dir_path in [('genomes', self.dirs['genomes']),
                              ('proteins', self.dirs['proteins']),
                              ('cds', self.dirs['cds']),  # 👈 新增
                              ('gff', self.dirs['gff'])]:
            if os.path.exists(dir_path):
                files = sorted([f for f in os.listdir(dir_path)
                              if not f.startswith('.')])
                lists[key] = [os.path.join(dir_path, f) for f in files]

        # 保存列表
        for key, file_list in lists.items():
            list_file = os.path.join(self.dirs['metadata'], f'{key}_list.txt')
            with open(list_file, 'w') as f:
                f.write('\n'.join(file_list))
            self.logger.info(f"{key.capitalize()} 列表: {list_file} ({len(file_list)} 个文件)")

        return lists

    def print_summary(self):
        """打印摘要"""
        self.logger.info("\n" + "="*60)
        self.logger.info("文件整理完成!")
        self.logger.info(f"输出目录: {self.output_dir}/")
        self.logger.info("\n目录结构:")
        self.logger.info(f"  genomes/   - 基因组序列 (.fna)")
        self.logger.info(f"  proteins/  - 蛋白质序列 (.faa)")
        self.logger.info(f"  cds/       - CDS核苷酸序列 (.fna)")  # 👈 新增
        self.logger.info(f"  gff/       - 基因注释 (.gff)")
        self.logger.info(f"  metadata/  - 元数据和文件列表")
        self.logger.info(f"  logs/      - 日志文件")
        self.logger.info("="*60)


def main():
    # 配置
    DOWNLOAD_DIR = "cglutamicum_downloads"  # 下载脚本的输出目录
    OUTPUT_DIR = "cglutamicum_pangenome"     # 整理后的输出目录

    print("="*60)
    print("C. glutamicum 基因组数据提取器")
    print("="*60)

    # 检查下载目录
    if not os.path.exists(DOWNLOAD_DIR):
        print(f"错误: 下载目录不存在: {DOWNLOAD_DIR}")
        print("请确保已运行下载脚本并完成下载")
        return

    # 创建提取器
    extractor = GenomeExtractor(DOWNLOAD_DIR, OUTPUT_DIR)

    # 查找所有基因组文件
    genome_zips = extractor.find_all_genome_zips()

    if not genome_zips:
        print("未找到基因组文件!")
        return

    print(f"\n找到 {len(genome_zips)} 个基因组文件")

    # 提取所有基因组
    successful, failed = extractor.extract_all_genomes(genome_zips)

    # 生成文件列表
    extractor.generate_file_lists()

    # 打印摘要
    extractor.print_summary()

if __name__ == "__main__":
    main()

2025-12-19 10:19:23,848 - INFO - 日志文件: cglutamicum_pangenome\logs\extract_20251219_101923.log
2025-12-19 10:19:23,849 - INFO - 扫描下载目录: cglutamicum_downloads
2025-12-19 10:19:23,850 - INFO - 找到 34 个基因组文件
2025-12-19 10:19:23,850 - INFO - 开始提取基因组文件...


C. glutamicum 基因组数据提取器

找到 34 个基因组文件


解压基因组: 100%|██████████| 34/34 [00:00<00:00, 39.91genome/s]
2025-12-19 10:19:24,704 - INFO - 
提取完成: 34/34 成功
2025-12-19 10:19:25,116 - INFO - 元数据摘要: cglutamicum_pangenome\metadata\genomes_summary.csv
2025-12-19 10:19:25,116 - INFO - 
=== 基因组统计 ===
2025-12-19 10:19:25,117 - INFO - 总数: 34
2025-12-19 10:19:25,119 - INFO - 有GFF注释: 34
2025-12-19 10:19:25,119 - INFO - 有蛋白序列: 34
2025-12-19 10:19:25,120 - INFO - 有CDS序列: 34
2025-12-19 10:19:25,121 - INFO - 
组装水平分布:
2025-12-19 10:19:25,124 - INFO -   Complete Genome: 34
2025-12-19 10:19:25,126 - INFO - Genomes 列表: cglutamicum_pangenome\metadata\genomes_list.txt (34 个文件)
2025-12-19 10:19:25,127 - INFO - Proteins 列表: cglutamicum_pangenome\metadata\proteins_list.txt (34 个文件)
2025-12-19 10:19:25,128 - INFO - Cds 列表: cglutamicum_pangenome\metadata\cds_list.txt (34 个文件)
2025-12-19 10:19:25,129 - INFO - Gff 列表: cglutamicum_pangenome\metadata\gff_list.txt (34 个文件)
2025-12-19 10:19:25,129 - INFO - 
2025-12-19 10:19:25,129 - INFO - 文件整理完成!
2025-12-19 10:19

# 建树-在WSL中运行

In [ ]:
"""
conda info --envs
cd /mnt/d/pycharm/iCG67_zxy/
python script/run_complete_pipeline.py \
    -i cglutamicum_pangenome/genomes \
    -o snp_tree_results_docker \
    -t 8 \
    --species "Corynebacterium glutamicum" \
    --ani-threshold 100
"""

# 识别同源基因-OrthoFinder in WSL

In [ ]:
# conda create -n orthofinder_py310 python=3.10 -y
# conda activate orthofinder_py310
# conda install -c bioconda -c conda-forge orthofinder -y
# cd /mnt/d/pycharm/iCG67_zxy/
#
#运行 OrthoFinder
'''
  orthofinder -f cglutamicum_pangenome/proteins \
              -t 14 \
              -a 8 \
              -S diamond \
              -o orthofinder_results
'''

# 对映locu_tag

In [ ]:
# ============================================================
# 【只需修改这里的路径】
# ============================================================
INPUT_FILE = "orthofinder_results/Results_Dec14/Orthogroups/Orthogroups.tsv"
GENOMES_DIR = "cglutamicum_pangenome/genomes"
CDS_DIR = "cglutamicum_pangenome/cds" # 如果CDS文件在单独目录则填写路径
OUTPUT_FILE = "converted_orthogroups.tsv"

# ============================================================
# 以下代码无需修改
# ============================================================
import os, re
import pandas as pd
from pathlib import Path

def parse_fna_for_species_name(fna_file):
    with open(fna_file, 'r') as f:
        first_line = f.readline().strip()
    if not first_line.startswith('>'): return None
    parts = first_line[1:].split(' ', 1)
    if len(parts) < 2: return None
    desc = re.sub(r',?\s*(complete\s+)?(sequence|genome|chromosome).*$', '', parts[1], flags=re.IGNORECASE)
    return desc.strip() if desc.strip() else None

def parse_cds_file(cds_file):
    wp_to_locus = {}
    with open(cds_file, 'r') as f:
        for line in f:
            if not line.startswith('>'): continue
            wp_match = re.search(r'(WP_\d+\.\d+)', line)
            locus_match = re.search(r'\[locus_tag=([^\]]+)\]', line)
            if wp_match and locus_match:
                wp_to_locus[wp_match.group(1)] = locus_match.group(1)
    return wp_to_locus

def convert_cell(cell_value, wp_to_locus):
    if pd.isna(cell_value) or str(cell_value).strip() in ['', '<null>']: return cell_value
    ids = re.split(r',\s*', str(cell_value))
    return ', '.join([wp_to_locus.get(i.strip(), i.strip()) for i in ids if i.strip()])

# 查找文件
genomes_dir = Path(GENOMES_DIR)
cds_dir = Path(CDS_DIR) if CDS_DIR else genomes_dir

fna_files = {f.stem: f for f in genomes_dir.glob('*.fna')}
print(f"找到 {len(fna_files)} 个.fna文件")

cds_files = {}
for pattern in ['*_cds_from_genomic.fna', '*.faa', '*_cds.fna']:
    for f in cds_dir.glob(pattern):
        for gcf_id in fna_files:
            if gcf_id in str(f):
                cds_files[gcf_id] = f
print(f"找到 {len(cds_files)} 个CDS文件")

# 【关键修正】为每个菌株单独构建映射
gcf_to_species = {}
gcf_to_wp_locus = {}  # {gcf_id: {wp_id: locus_tag}}

for gcf_id, fna_path in fna_files.items():
    species = parse_fna_for_species_name(fna_path)
    gcf_to_species[gcf_id] = species if species else gcf_id

    if gcf_id in cds_files:
        gcf_to_wp_locus[gcf_id] = parse_cds_file(cds_files[gcf_id])
        print(f"{gcf_id} -> {gcf_to_species[gcf_id]} ({len(gcf_to_wp_locus[gcf_id])} 映射)")
    else:
        gcf_to_wp_locus[gcf_id] = {}
        print(f"{gcf_id} -> {gcf_to_species[gcf_id]} [无CDS文件]")

# 读取数据
df = pd.read_csv(INPUT_FILE, sep='\t')
print(f"\n原始数据: {len(df)} 行 x {len(df.columns)} 列")

# 保存原始列名到GCF的映射
original_cols = list(df.columns)

# 【关键】转换每列时使用对应菌株的映射表
for col_idx, gcf_id in enumerate(original_cols[1:], 1):
    strain_mapping = gcf_to_wp_locus.get(gcf_id, {})
    df.iloc[:, col_idx] = df.iloc[:, col_idx].apply(lambda x: convert_cell(x, strain_mapping))

# 转换列名
df.columns = ['Orthogroup'] + [gcf_to_species.get(c, c) for c in original_cols[1:]]

# 保存
df.to_csv(OUTPUT_FILE, sep='\t', index=False)
print(f"\n✅ 完成! 已保存到: {OUTPUT_FILE}")

# 分析同源以及非同源基因

In [1]:
#!/usr/bin/env python3
"""
OrthoFinder Ortholog Analysis - Streamlined Output
Analysis of orthologous genes between C. glutamicum ATCC 14067 and ATCC 13032

Outputs:
1. Comprehensive text report (analysis_report.txt)
2. Single Excel file with multiple sheets (ortholog_data.xlsx)
"""

import pandas as pd
import os


def parse_orthogroups(filepath, strain1_keyword="14067", strain2_keyword="13032"):
    """
    Parse OrthoFinder Orthogroups.tsv file
    """
    df = pd.read_csv(filepath, sep='\t')

    # Find columns for each strain
    strain1_cols = [col for col in df.columns if strain1_keyword in col]
    strain2_cols = [col for col in df.columns if strain2_keyword in col]

    if not strain1_cols or not strain2_cols:
        print(f"Available columns: {df.columns.tolist()}")
        raise ValueError(f"Could not find columns containing '{strain1_keyword}' or '{strain2_keyword}'")

    strain1_col = strain1_cols[0]
    strain2_col = strain2_cols[0]

    print(f"Analyzing: {strain1_col} vs {strain2_col}")

    # Initialize
    strain1_genes = set()
    strain2_genes = set()
    shared_orthogroups = []
    strain1_only_orthogroups = []
    strain2_only_orthogroups = []

    # Process each orthogroup
    for idx, row in df.iterrows():
        og = row['Orthogroup']

        s1_genes_str = str(row[strain1_col]) if pd.notna(row[strain1_col]) else ""
        s2_genes_str = str(row[strain2_col]) if pd.notna(row[strain2_col]) else ""

        s1_genes = set()
        s2_genes = set()

        if s1_genes_str and s1_genes_str != 'nan':
            s1_genes = set([g.strip() for g in s1_genes_str.replace(',', ' ').split() if g.strip()])

        if s2_genes_str and s2_genes_str != 'nan':
            s2_genes = set([g.strip() for g in s2_genes_str.replace(',', ' ').split() if g.strip()])

        strain1_genes.update(s1_genes)
        strain2_genes.update(s2_genes)

        has_s1 = len(s1_genes) > 0
        has_s2 = len(s2_genes) > 0

        if has_s1 and has_s2:
            shared_orthogroups.append({
                'orthogroup': og,
                'strain1_genes': s1_genes,
                'strain2_genes': s2_genes,
                'strain1_count': len(s1_genes),
                'strain2_count': len(s2_genes)
            })
        elif has_s1:
            strain1_only_orthogroups.append({
                'orthogroup': og,
                'genes': s1_genes,
                'count': len(s1_genes)
            })
        elif has_s2:
            strain2_only_orthogroups.append({
                'orthogroup': og,
                'genes': s2_genes,
                'count': len(s2_genes)
            })

    shared_genes_s1 = sum([og['strain1_count'] for og in shared_orthogroups])
    shared_genes_s2 = sum([og['strain2_count'] for og in shared_orthogroups])
    unique_genes_s1 = sum([og['count'] for og in strain1_only_orthogroups])
    unique_genes_s2 = sum([og['count'] for og in strain2_only_orthogroups])

    return {
        'strain1_name': strain1_col,
        'strain2_name': strain2_col,
        'strain1_total_genes': len(strain1_genes),
        'strain2_total_genes': len(strain2_genes),
        'shared_orthogroups': shared_orthogroups,
        'strain1_only_orthogroups': strain1_only_orthogroups,
        'strain2_only_orthogroups': strain2_only_orthogroups,
        'num_shared_orthogroups': len(shared_orthogroups),
        'num_strain1_only_orthogroups': len(strain1_only_orthogroups),
        'num_strain2_only_orthogroups': len(strain2_only_orthogroups),
        'shared_genes_s1': shared_genes_s1,
        'shared_genes_s2': shared_genes_s2,
        'unique_genes_s1': unique_genes_s1,
        'unique_genes_s2': unique_genes_s2
    }


def generate_comprehensive_report(results, output_path):
    """
    Generate comprehensive analysis report with data preview
    """
    s1_short = "ATCC 14067"
    s2_short = "ATCC 13032"

    report = []
    report.append("=" * 80)
    report.append("ORTHOFINDER COMPARATIVE ANALYSIS REPORT")
    report.append("Corynebacterium glutamicum ATCC 14067 vs ATCC 13032")
    report.append("=" * 80)
    report.append("")

    # Summary Statistics
    report.append("1. SUMMARY STATISTICS")
    report.append("-" * 80)
    report.append(f"{'Metric':<50} {'Value':>15}")
    report.append("-" * 80)
    report.append(f"{'Total genes in ' + s1_short:<50} {results['strain1_total_genes']:>15,}")
    report.append(f"{'Total genes in ' + s2_short:<50} {results['strain2_total_genes']:>15,}")
    report.append("")

    # Orthogroup Analysis
    report.append("2. ORTHOGROUP ANALYSIS")
    report.append("-" * 80)
    total_ogs = (results['num_shared_orthogroups'] +
                 results['num_strain1_only_orthogroups'] +
                 results['num_strain2_only_orthogroups'])

    report.append(f"{'Total orthogroups analyzed:':<50} {total_ogs:>15,}")
    report.append(f"{'Shared orthogroups (core genome):':<50} {results['num_shared_orthogroups']:>15,}")
    report.append(f"{'Orthogroups unique to ' + s1_short + ':':<50} {results['num_strain1_only_orthogroups']:>15,}")
    report.append(f"{'Orthogroups unique to ' + s2_short + ':':<50} {results['num_strain2_only_orthogroups']:>15,}")

    if total_ogs > 0:
        report.append(f"{'Shared orthogroups (%):':<50} {results['num_shared_orthogroups']/total_ogs*100:>14.1f}%")
    report.append("")

    # Gene Distribution
    report.append("3. GENE DISTRIBUTION")
    report.append("-" * 80)
    report.append(f"{'Core genes (' + s1_short + '):':<50} {results['shared_genes_s1']:>15,}")
    report.append(f"{'Core genes (' + s2_short + '):':<50} {results['shared_genes_s2']:>15,}")
    report.append(f"{'Strain-specific genes (' + s1_short + '):':<50} {results['unique_genes_s1']:>15,}")
    report.append(f"{'Strain-specific genes (' + s2_short + '):':<50} {results['unique_genes_s2']:>15,}")
    report.append("")

    # Core Genome Analysis
    if results['strain1_total_genes'] > 0:
        core_pct_s1 = results['shared_genes_s1'] / results['strain1_total_genes'] * 100
    else:
        core_pct_s1 = 0

    if results['strain2_total_genes'] > 0:
        core_pct_s2 = results['shared_genes_s2'] / results['strain2_total_genes'] * 100
    else:
        core_pct_s2 = 0

    report.append("4. CORE GENOME ANALYSIS")
    report.append("-" * 80)
    report.append(f"{'Core genome proportion (' + s1_short + '):':<50} {core_pct_s1:>14.1f}%")
    report.append(f"{'Core genome proportion (' + s2_short + '):':<50} {core_pct_s2:>14.1f}%")
    report.append("")

    # Data Preview - Shared Orthogroups
    report.append("5. DATA PREVIEW - SHARED ORTHOGROUPS (First 10)")
    report.append("-" * 80)
    for i, og in enumerate(results['shared_orthogroups'][:10], 1):
        report.append(f"\n{i}. Orthogroup: {og['orthogroup']}")
        report.append(f"   {s1_short} genes ({og['strain1_count']}): {', '.join(sorted(list(og['strain1_genes'])[:5]))}")
        if og['strain1_count'] > 5:
            report.append(f"      ... and {og['strain1_count'] - 5} more")
        report.append(f"   {s2_short} genes ({og['strain2_count']}): {', '.join(sorted(list(og['strain2_genes'])[:5]))}")
        if og['strain2_count'] > 5:
            report.append(f"      ... and {og['strain2_count'] - 5} more")
    report.append("")

    # Data Preview - Strain-specific
    report.append("6. DATA PREVIEW - STRAIN-SPECIFIC ORTHOGROUPS (First 5 each)")
    report.append("-" * 80)
    report.append(f"\n{s1_short}-specific:")
    for i, og in enumerate(results['strain1_only_orthogroups'][:5], 1):
        genes_list = ', '.join(sorted(list(og['genes'])[:5]))
        report.append(f"   {i}. {og['orthogroup']} ({og['count']} genes): {genes_list}")
        if og['count'] > 5:
            report.append(f"      ... and {og['count'] - 5} more")

    report.append(f"\n{s2_short}-specific:")
    for i, og in enumerate(results['strain2_only_orthogroups'][:5], 1):
        genes_list = ', '.join(sorted(list(og['genes'])[:5]))
        report.append(f"   {i}. {og['orthogroup']} ({og['count']} genes): {genes_list}")
        if og['count'] > 5:
            report.append(f"      ... and {og['count'] - 5} more")
    report.append("")

    # File Reference
    report.append("7. OUTPUT FILES")
    report.append("-" * 80)
    report.append("Complete gene lists exported to: ortholog_data.xlsx")
    report.append("  - Sheet 1: Summary Statistics")
    report.append("  - Sheet 2: Shared Orthogroups")
    report.append("  - Sheet 3: ATCC 14067-specific Orthogroups")
    report.append("  - Sheet 4: ATCC 13032-specific Orthogroups")
    report.append("")

    report.append("=" * 80)
    report.append("End of Report")
    report.append("=" * 80)

    report_text = "\n".join(report)

    with open(output_path, 'w') as f:
        f.write(report_text)

    print(report_text)
    print(f"\nReport saved to: {output_path}")

    return report_text


def export_to_excel(results, output_path):
    """
    Export all data to a single Excel file with multiple sheets
    """
    s1_short = "ATCC_14067"
    s2_short = "ATCC_13032"

    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Sheet 1: Summary Statistics
        summary_data = {
            'Metric': [
                'Total Genes (ATCC 14067)',
                'Total Genes (ATCC 13032)',
                '',
                'Total Orthogroups',
                'Shared Orthogroups',
                'ATCC 14067-specific Orthogroups',
                'ATCC 13032-specific Orthogroups',
                '',
                'Core Genes in ATCC 14067',
                'Core Genes in ATCC 13032',
                'Unique Genes in ATCC 14067',
                'Unique Genes in ATCC 13032',
                '',
                'Core Genome % (ATCC 14067)',
                'Core Genome % (ATCC 13032)'
            ],
            'Value': [
                results['strain1_total_genes'],
                results['strain2_total_genes'],
                '',
                results['num_shared_orthogroups'] + results['num_strain1_only_orthogroups'] + results['num_strain2_only_orthogroups'],
                results['num_shared_orthogroups'],
                results['num_strain1_only_orthogroups'],
                results['num_strain2_only_orthogroups'],
                '',
                results['shared_genes_s1'],
                results['shared_genes_s2'],
                results['unique_genes_s1'],
                results['unique_genes_s2'],
                '',
                f"{results['shared_genes_s1']/results['strain1_total_genes']*100:.1f}%" if results['strain1_total_genes'] > 0 else 'N/A',
                f"{results['shared_genes_s2']/results['strain2_total_genes']*100:.1f}%" if results['strain2_total_genes'] > 0 else 'N/A'
            ]
        }
        df_summary = pd.DataFrame(summary_data)
        df_summary.to_excel(writer, sheet_name='Summary', index=False)

        # Sheet 2: Shared Orthogroups
        shared_data = []
        for og in results['shared_orthogroups']:
            shared_data.append({
                'Orthogroup': og['orthogroup'],
                f'{s1_short}_Genes': ', '.join(sorted(og['strain1_genes'])),
                f'{s1_short}_Count': og['strain1_count'],
                f'{s2_short}_Genes': ', '.join(sorted(og['strain2_genes'])),
                f'{s2_short}_Count': og['strain2_count']
            })

        if shared_data:
            df_shared = pd.DataFrame(shared_data)
            df_shared.to_excel(writer, sheet_name='Shared_Orthogroups', index=False)
            print(f"Exported {len(shared_data)} shared orthogroups")

        # Sheet 3: ATCC 14067-specific
        s1_data = []
        for og in results['strain1_only_orthogroups']:
            s1_data.append({
                'Orthogroup': og['orthogroup'],
                'Genes': ', '.join(sorted(og['genes'])),
                'Gene_Count': og['count']
            })

        if s1_data:
            df_s1 = pd.DataFrame(s1_data)
            df_s1.to_excel(writer, sheet_name='ATCC14067_Specific', index=False)
            print(f"Exported {len(s1_data)} ATCC 14067-specific orthogroups")

        # Sheet 4: ATCC 13032-specific
        s2_data = []
        for og in results['strain2_only_orthogroups']:
            s2_data.append({
                'Orthogroup': og['orthogroup'],
                'Genes': ', '.join(sorted(og['genes'])),
                'Gene_Count': og['count']
            })

        if s2_data:
            df_s2 = pd.DataFrame(s2_data)
            df_s2.to_excel(writer, sheet_name='ATCC13032_Specific', index=False)
            print(f"Exported {len(s2_data)} ATCC 13032-specific orthogroups")

    print(f"\nAll data exported to: {output_path}")


def main():
    """Main function"""
    input_file = "converted_orthogroups.tsv"
    output_dir = "./orthooutput"

    os.makedirs(output_dir, exist_ok=True)

    print("=" * 80)
    print("OrthoFinder Comparative Analysis")
    print("C. glutamicum ATCC 14067 vs ATCC 13032")
    print("=" * 80)
    print()

    print("Parsing OrthoFinder results...")
    results = parse_orthogroups(input_file, "14067", "13032")

    print("\nGenerating comprehensive report...")
    generate_comprehensive_report(results, os.path.join(output_dir, "analysis_report.txt"))

    print("\nExporting all data to Excel...")
    export_to_excel(results, os.path.join(output_dir, "ortholog_data.xlsx"))

    print("\n" + "=" * 80)
    print("Analysis complete!")
    print("=" * 80)
    print("\nOutput files:")
    print("  1. analysis_report.txt  - Comprehensive text report with statistics and previews")
    print("  2. ortholog_data.xlsx   - Excel file with 4 sheets containing all gene data")
    print("=" * 80)


if __name__ == "__main__":
    main()

OrthoFinder Comparative Analysis
C. glutamicum ATCC 14067 vs ATCC 13032

Parsing OrthoFinder results...
Analyzing: Corynebacterium glutamicum ATCC 14067 vs Corynebacterium glutamicum ATCC 13032

Generating comprehensive report...
ORTHOFINDER COMPARATIVE ANALYSIS REPORT
Corynebacterium glutamicum ATCC 14067 vs ATCC 13032

1. SUMMARY STATISTICS
--------------------------------------------------------------------------------
Metric                                                       Value
--------------------------------------------------------------------------------
Total genes in ATCC 14067                                    3,029
Total genes in ATCC 13032                                    2,941

2. ORTHOGROUP ANALYSIS
--------------------------------------------------------------------------------
Total orthogroups analyzed:                                  3,306
Shared orthogroups (core genome):                            2,570
Orthogroups unique to ATCC 14067:                    